Robust clustering

In [3]:
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt

## Check clustering result 

In [4]:
with open("./clustering.pkl", "rb") as fp:
    cluster_result = pkl.load(fp)
cluster_result.keys()

N = cluster_result["km_objs"][0].cluster_centers_.shape[0]

In [30]:
adj_mat = cluster_result["same_prob"]
degree = [np.sum(adj_mat[i]) for i in range(adj_mat.shape[0])]

In [139]:
with open("./data/network.pkl", "wb") as fp:
    pkl.dump({"adj_mat": adj_mat,
              "degree": degree}, fp)

## Generate network 

In [36]:
import networkx as nx

# set attribute
with open("../simulation_data/purified_data.pkl", "rb") as fp:
    pdata = pkl.load(fp)
    
with open("./data/rcluster_nrank3.pkl", "rb") as fp:
    cinfo = pkl.load(fp)

In [164]:
pdata.keys()

dict_keys(['data', 'row_labels', 'row_names', 'col_names', 'removed'])

In [175]:
# Show random 1000 points
np.random.seed(100)

nshow = 500
p = np.array(degree)/np.sum(degree)
node_id = np.random.choice(len(p), size=nshow, p=p, replace=False)
edges = [(i, j, adj_mat[i, j]) for i in node_id for j in node_id if i != j]

In [177]:
def set_node_attrib(G, attrib_val, attrib_name):
    attrib = dict()
    for n in range(len(attrib_val)):
        attrib[n] = attrib_val[n]
    nx.set_node_attributes(G, attrib, attrib_name)
    
G = nx.Graph()
G.add_nodes_from(node_id)
G.add_weighted_edges_from(edges)

set_node_attrib(G, cinfo["rcluster_id"], "clsuter_id")

for n in range(pdata["data"].shape[0]):
    set_node_attrib(G, pdata["data"][n, :], pdata["row_names"][n])
    
for n in range(len(pdata["removed"]["row_names"])):
    set_node_attrib(G, pdata["removed"]["data"][n, :], pdata["removed"]["row_names"][n])

In [187]:
nx.write_gexf(G, "graph.gexf")